<a href="https://colab.research.google.com/github/amihlyaeva/python/blob/main/Geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Импортировать необходимые модули
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import random
import numpy as np

In [ ]:
#Загрузка границ участка и ограничений из GeoJSON
def load_site_and_restrictions(geojson_file):
    gdf = gpd.read_file(geojson_file)

    # Получаем границы участка (Polygon) – допустим, они всегда первые
    site_boundary = gdf.geometry.iloc[0]  # Зона застройки (Polygon)

    # Остальные объекты – ограничения
    restrictions = gdf.geometry[1:]  # Все остальные объекты (Polygon или LineString)

    return site_boundary, restrictions

In [ ]:
#размещения объектов

def auto_building_with_roads(geojson_file, density_limit, min_distance, road_width):
    # Загрузить границы участка и ограничения
    site_polygon, restrictions = load_site_and_restrictions(geojson_file)

    # Пространство для построенных объектов
    buildings = []
    parks = []
    roads = []

    # Общая площадь участка и зона застройки
    area = site_polygon.area
    buildable_area = area * (density_limit / 100)

    # Генерация парков
    num_parks = 1  # Указываем количество парков, можно изменить на произвольное
    for _ in range(num_parks):
        park_location = Point(random.uniform(site_polygon.bounds[0], site_polygon.bounds[2]),
                              random.uniform(site_polygon.bounds[1], site_polygon.bounds[3]))
        park_radius = random.uniform(10, 20)  # Задайте радиус парка
        park = park_location.buffer(park_radius)

        while not site_polygon.contains(park) or restrictions.geometry.intersects(park).any():
            park_location = Point(random.uniform(site_polygon.bounds[0], site_polygon.bounds[2]),
                                  random.uniform(site_polygon.bounds[1], site_polygon.bounds[3]))
            park = park_location.buffer(park_radius)

        parks.append(park)

    # Генерация зданий
    while buildable_area > 0:
        x = random.uniform(site_polygon.bounds[0], site_polygon.bounds[2])
        y = random.uniform(site_polygon.bounds[1], site_polygon.bounds[3])
        random_point = Point(x, y)

        building_radius = random.uniform(5, 15)
        building = random_point.buffer(building_radius)

        if not site_polygon.contains(building) or restrictions.geometry.intersects(building).any():
            continue

        if any(building.distance(Point(b.centroid.x, b.centroid.y)) < min_distance for b in buildings + parks):
            continue

        buildings.append(building)
        buildable_area -= building.area

    # Генерация дорог
    for _ in range(5):  # Указываем количество дорог, можно изменить
        start = Point(random.uniform(site_polygon.bounds[0], site_polygon.bounds[2]),
                      random.uniform(site_polygon.bounds[1], site_polygon.bounds[3]))
        end = Point(random.uniform(site_polygon.bounds[0], site_polygon.bounds[2]),
                    random.uniform(site_polygon.bounds[1], site_polygon.bounds[3]))

        # Создание линии дороги
        road_line = gpd.GeoSeries([LineString([start, end])]).buffer(road_width / 2).unary_union

        if not site_polygon.contains(road_line) or restrictions.geometry.intersects(road_line).any():
            continue

        roads.append(road_line)

    # Конвертируем в GeoDataFrame
    buildings_gdf = gpd.GeoDataFrame(geometry=buildings)
    parks_gdf = gpd.GeoDataFrame(geometry=parks)
    roads_gdf = gpd.GeoDataFrame(geometry=roads)

    return buildings_gdf, parks_gdf, roads_gdf

In [ ]:
#генерация плана
def generate_complete_plan_image(buildings_df, parks_df, roads_df, output_image='complete_plan.png'):
    plt.figure(figsize=(10, 10))

    # Отрисовка дорог
    if not roads_df.empty:
        roads_df.plot(ax=plt.gca(), color='brown', alpha=0.7, edgecolor='black', label='Roads')

    # Отрисовка зданий
    if not buildings_df.empty:
        buildings_df.plot(ax=plt.gca(), color='lightblue', alpha=0.7, edgecolor='black', label='Buildings')

    # Отрисовка парков
    if not parks_df.empty:
        parks_df.plot(ax=plt.gca(), color='green', alpha=0.5, edgecolor='black', label='Parks')

    plt.title("Site Plan with Buildings, Parks, and Roads")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.legend()
    plt.grid()
    plt.savefig(output_image)
    plt.close()
    # Отображение плана на экране
    plt.show()
    plt.close()  # Закройте текущее изображение

In [ ]:
geojson_file = '/content/input.geojson'  # Путь к вашему входному GeoJSON файлу
density_limit = 30  # Процент площади участка для застройки
min_distance = 10  # Минимальное расстояние между объектами
road_width = 5  # Ширина дороги

# Запускаем алгоритм
buildings, parks, roads = auto_building_with_roads(geojson_file, density_limit, min_distance, road_width)

# Сохранение результатов в GeoJSON
buildings.to_file('output_buildings.geojson', driver='GeoJSON')
parks.to_file('output_parks.geojson', driver='GeoJSON')
roads.to_file('output_roads.geojson', driver='GeoJSON')

# Сгенерировать изображение плана участка
generate_complete_plan_image(buildings, parks, roads)